In [92]:
!pip install catboost

import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier, VotingClassifier, RandomForestClassifier,StackingClassifier,BaggingClassifier,AdaBoostClassifier,ExtraTreesClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, RobustScaler, OneHotEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, classification_report, make_scorer,mean_squared_error
from catboost import CatBoostClassifier, Pool, cv
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.naive_bayes import GaussianNB, CategoricalNB
from sklearn.metrics import roc_curve, roc_auc_score, auc
# import matplotlib.pyplot as plt
# import lightgbm as lgb
import xgboost as xgb


In [93]:

train_set=pd.read_csv('train_set.csv')
test_set=pd.read_csv('test_set.csv')

In [94]:
categorical_cols = []
numerical_cols = []

for column in train_set.columns:
  if train_set[column].dtype == object or train_set[column].nunique() < 10:
    categorical_cols.append(column)
  else:
    numerical_cols.append(column)

print("Categorical Columns:", categorical_cols)
print("Numerical Columns:", numerical_cols)

Categorical Columns: ['X4', 'X5', 'X6', 'X8', 'X10', 'X11', 'X16', 'Y']
Numerical Columns: ['RecordId', 'X2', 'X3', 'X7', 'X9', 'X12', 'X13', 'X14', 'X15', 'X17', 'X18', 'X19', 'X20', 'X21', 'X22', 'X23', 'X24', 'X25', 'X26', 'X27', 'X28', 'X29', 'X30', 'X31', 'X32', 'X33', 'X34', 'X35', 'X36', 'X37', 'X38', 'X39', 'X40', 'X41', 'X42', 'X43', 'X44', 'X45', 'X46', 'X47', 'X48', 'X49', 'X50', 'X51', 'X52', 'X53', 'X54', 'X55', 'X56', 'X57', 'X58', 'X59', 'X60', 'X61', 'X62', 'X63', 'X64', 'X65', 'X66', 'X67', 'X68', 'X69', 'X70', 'X71', 'X72', 'X73', 'X74', 'X75', 'X76', 'X77', 'X78']


In [95]:
# Handle missing values for both train and test datasets
for column in categorical_cols:
  if column in train_set.columns and train_set[column].isnull().any():
    mode_imputer = SimpleImputer(strategy='most_frequent')
    train_set[column] = mode_imputer.fit_transform(train_set[[column]])
  if column in test_set.columns and test_set[column].isnull().any():
    if column in train_set.columns:
      mode_imputer = SimpleImputer(strategy='most_frequent')
      test_set[column] = mode_imputer.fit_transform(test_set[[column]])
    else:
      print(f"Warning: Column '{column}' is missing in the training set and cannot be imputed in the test set.")

for column in numerical_cols:
  if column in train_set.columns and train_set[column].isnull().any():
    mean_imputer = SimpleImputer(strategy='mean')
    train_set[column] = mean_imputer.fit_transform(train_set[[column]])
  if column in test_set.columns and test_set[column].isnull().any():
    if column in train_set.columns:
      mean_imputer = SimpleImputer(strategy='mean')
      test_set[column] = mean_imputer.fit_transform(test_set[[column]])
    else:
      print(f"Warning: Column '{column}' is missing in the training set and cannot be imputed in the test set.")


# Find missing values in the training set
missing_values = train_set.isnull().sum()
print(missing_values[missing_values > 0])

# Find missing values in the test set
missing_values = test_set.isnull().sum()
print(missing_values[missing_values > 0])



Series([], dtype: int64)
Series([], dtype: int64)


In [96]:


# Assuming train_set and test_set are pandas DataFrames
# Get all columns except 'Y' for X
X = train_set[[col for col in train_set.columns if col != 'Y']]

# Get only 'Y' column for y
y = train_set['Y']

# Select the same features for the test data
X_testdata = test_set[[col for col in test_set.columns if col != 'Y']]
if 'RecordId' in X.columns:
  X = X.drop('RecordId', axis=1)
if 'RecordId' in X_testdata.columns:
  X_testdata = X_testdata.drop('RecordId', axis=1)

# ... rest of your code (scaling, feature selection, model training, etc.) ...
X.columns

Index(['X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11', 'X12',
       'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20', 'X21', 'X22',
       'X23', 'X24', 'X25', 'X26', 'X27', 'X28', 'X29', 'X30', 'X31', 'X32',
       'X33', 'X34', 'X35', 'X36', 'X37', 'X38', 'X39', 'X40', 'X41', 'X42',
       'X43', 'X44', 'X45', 'X46', 'X47', 'X48', 'X49', 'X50', 'X51', 'X52',
       'X53', 'X54', 'X55', 'X56', 'X57', 'X58', 'X59', 'X60', 'X61', 'X62',
       'X63', 'X64', 'X65', 'X66', 'X67', 'X68', 'X69', 'X70', 'X71', 'X72',
       'X73', 'X74', 'X75', 'X76', 'X77', 'X78'],
      dtype='object')

In [97]:
# scalar=MinMaxScaler()
# X = scalar.fit_transform(X)
# X_testdata = scalar.transform(X_testdata)




In [98]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [99]:
# from sklearn.model_selection import RandomizedSearchCV
# import xgboost as xgb

# # Define the parameter grid
# param_grid = {
#     'learning_rate': [0.01, 0.03, 0.1],
#     'max_depth': [4, 6, 8],
#     'n_estimators': [100, 200, 376,500],
#     'gamma': [0, 0.1, 0.2],
#     'reg_lambda': [0.01, 0.1, 1],
#     'min_child_weight': [1, 3, 5],
#     'colsample_bytree': [0.5, 0.7, 1.0],
#     'colsample_bylevel': [0.5, 0.7, 1.0],
#     'scale_pos_weight': [1, 1.1, 1.2],
#     'subsample': [0.7, 0.8, 1.0]
# }

# # Initialize the classifier
# clf = xgb.XGBClassifier(
#     objective='binary:logistic',
#     eval_metric='logloss',
#     random_state=42
# )

# random_search = RandomizedSearchCV(estimator=clf, param_distributions=param_grid, cv=3, n_iter=50, scoring='roc_auc', verbose=1, n_jobs=-1)

# # Fit the model
# random_search.fit(X_train, y_train)

# # Print the best parameters
# print("Best parameters found: ", random_search.best_params_)

In [100]:
# # prompt: like the following geneerate good params for catboost but stire separately:import xgboost as xgb
# # from sklearn.model_selection import GridSearchCV
# # # Define the parameter grid
# # param_grid = {
# #     'learning_rate': [0.01, 0.03, 0.1],
# #     'max_depth': [4, 6, 8],
# #     'n_estimators': [100, 200, 376,500],
# #     'gamma': [0, 0.1, 0.2],
# #     'reg_lambda': [0.01, 0.1, 1],
# #     'min_child_weight': [1, 3, 5],
# #     'colsample_bytree': [0.5, 0.7, 1.0],
# #     'colsample_bylevel': [0.5, 0.7, 1.0],
# #     'scale_pos_weight': [1, 1.1, 1.2],
# #     'subsample': [0.7, 0.8, 1.0]
# # }
# # # Initialize the classifier
# # clf = xgb.XGBCla

# from catboost import CatBoostClassifier
# from sklearn.model_selection import train_test_split, GridSearchCV
# import json

# # Assuming X_train, y_train are defined from your previous code

# # Define the parameter grid for CatBoost
# param_grid = {
#     'iterations': [100, 200, 300],
#     'learning_rate': [0.01, 0.1, 0.2],
#     'depth': [4, 6, 8],
#     'l2_leaf_reg': [1, 3, 5],
#     'border_count': [32, 64, 128],
#     'random_strength': [1, 1.5, 2],
#     'bagging_temperature': [0, 1, 2],
# }

# # Initialize the CatBoost classifier
# clf2 = CatBoostClassifier(
#     loss_function='Logloss',
#     eval_metric='AUC',
#     task_type='GPU',  # Use GPU if available
#     random_seed=42
# )


# # Initialize GridSearchCV for CatBoost
# grid_search2 = GridSearchCV(estimator=clf2, param_grid=param_grid, cv=3, scoring='roc_auc', verbose=1, n_jobs=-1)

# # Fit the model
# grid_search2.fit(X_train, y_train)

# # Print the best parameters
# print("Best parameters found: ", grid_search2.best_params_)

# # Store the best parameters in a JSON file
# best_params2 = grid_search2.best_params_
# with open('best_catboost_params.json', 'w') as f:
#     json.dump(best_params2, f, indent=4)

In [101]:
clf =xgb.XGBClassifier(
      subsample=1.0,
      scale_pos_weight= 1.2,
      reg_lambda= 0.01,
      n_estimators= 500,
      min_child_weight= 5,
      max_depth= 6,
      learning_rate= 0.03,
      gamma= 0.2,
      colsample_bytree= 0.7,
      colsample_bylevel=0.7,
      objective='binary:logistic',
    eval_metric='logloss',
    random_state=42
    # learning_rate= 0.0334925,
    # max_depth=6,
    # n_estimators=376,
    # objective='binary:logistic',
    # eval_metric='logloss',
    # random_state=42,
    # gamma=0.097,
    # reg_lambda=0.0271605,
    # min_child_weight=4.166362,
    # colsample_bytree=0.673,
    # colsample_bylevel= 0.65,
    # scale_pos_weight= 1.103739,
    # subsample=0.7967162407706075
)

clf.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=0.7, colsample_bynode=None,
              colsample_bytree=0.7, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=0.2, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.03, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=None, min_child_weight=5, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=500,
              n_jobs=None, num_parallel_tree=None, random_state=42, ...)

In [102]:
# Get feature importances
feature_importances = clf.feature_importances_

# Create a DataFrame for feature importances
feature_importances_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': feature_importances
})

# Sort the DataFrame by importance
feature_importances_df = feature_importances_df.sort_values(by='Importance', ascending=False)

# Print the feature importances
print(feature_importances_df)

top_features = feature_importances_df['Feature'].head(30).tolist()

   Feature  Importance
67     X69    0.048787
24     X26    0.045313
68     X70    0.038832
16     X18    0.034205
18     X20    0.028426
..     ...         ...
45     X47    0.008196
75     X77    0.007628
14     X16    0.005982
74     X76    0.004512
69     X71    0.000000

[77 rows x 2 columns]


In [103]:
X_train_top = X_train[top_features]
X_test_top = X_test[top_features]
X_testdata_top = X_testdata[top_features]

# Re-train the model with the top features
clf.fit(X_train_top, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=0.7, colsample_bynode=None,
              colsample_bytree=0.7, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=0.2, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.03, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=None, min_child_weight=5, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=500,
              n_jobs=None, num_parallel_tree=None, random_state=42, ...)

In [104]:
y_pred_proba = clf.predict_proba(X_test_top)[:, 1]

# Calculate AUC
auc = roc_auc_score(y_test, y_pred_proba)
print(f"AUC: {auc}")

AUC: 0.9612392191516015


In [105]:
# Predict probabilities for the test set
y_pred_proba_testdata = clf.predict_proba(X_testdata_top)[:, 1]

# Create a DataFrame with RecordId and predicted probabilities
test_set['Y_probability'] = y_pred_proba_testdata

# Save the DataFrame to a CSV file
test_set[['RecordId', 'Y_probability']].to_csv('test_set_with_probabilities.csv', index=False)

print("Probabilities saved to test_set_with_probabilities.csv")

import pandas as pd

# Read the generated CSV file
csv_file_path = 'test_set_with_probabilities.csv'
test_set_with_probabilities = pd.read_csv(csv_file_path)

# Get unique values in the Y_probability column
unique_y_probabilities = test_set_with_probabilities['Y_probability'].unique()

# Print the unique values
print(unique_y_probabilities)
print(test_set_with_probabilities)

Probabilities saved to test_set_with_probabilities.csv
[6.2866180e-04 8.4856940e-04 2.4060720e-05 ... 1.0722403e-01 1.5144704e-04
 3.7468170e-05]
        RecordId  Y_probability
0         300001       0.000629
1         300002       0.000849
2         300003       0.000024
3         300004       0.001239
4         300005       0.000064
...          ...            ...
105477    405478       0.000033
105478    405479       0.107224
105479    405480       0.000151
105480    405481       0.000037
105481    405482       0.000127

[105482 rows x 2 columns]
